In [1]:
import nltk
import spacy
from spacy import displacy
from nltk import Tree
from nltk.tree import ParentedTree
import es_core_news_sm
import stanza
from spacy_stanza import StanzaLanguage

#nlp = es_core_news_sm.load()

#nlp = spacy.load('es_core_news_sm')

/home/estudiantes/.local/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'es_core_news_sm' (3.1.0) requires spaCy v3.1 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_ + '-' + node.dep_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_ + '-' + node.dep_

def to_nltk_tree1(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.tag_, [to_nltk_tree1(child) for child in node.children])
    else:
        return node.tag_

def crear_arbol(texto):
    document = nlp(texto)
    jc = list(document.sents)[0]
    arbol = to_nltk_tree1(jc.root)
    # arbol = ParentedTree.convert(arbol)
    arbol.pretty_print()
    return arbol

In [3]:
def rec1(A):
    if type(A) is nltk.Tree:
        cadena = A.label() 
        for i, B in enumerate(A):
            cadena += '(' + rec1(B) + ')'
        return cadena
    else:
        return A

In [4]:
def aplicacion(A, diccionario):
    if type(A) is nltk.Tree:
        funcion = diccionario[A.label()]
        argumentos = [aplicacion(x, diccionario) for x in A]
        return funcion(*argumentos)
    else:
        return diccionario[A]

In [5]:
from nltk import SnowballStemmer

In [6]:
def limpiar_texto(texto, stopwords):
#     token = nltk.RegexpTokenizer(r"\w+")
#     tokens = token.tokenize(texto)
#     lemmatizer = nltk.stem.WordNetLemmatizer()
    snlp = stanza.Pipeline(lang="es")
    nlp = StanzaLanguage(snlp)
    texto = nlp(texto)
    #lista_texto = texto.split()
    lista = [token.lemma_ for token in texto]
    lista = [x for x in lista if x not in stopwords]
    return ' '.join(lista)

In [2]:
stanza.download('es')

2022-03-03 09:22:27 INFO: Downloading default packages for language: es (Spanish)...


AssertionError: md5 for /home/estudiantes/stanza_resources/es/default.zip is 58e67356857d3b40cba98f68d1b26690, expected 1a63eaa21cad42f23d98a1e5c996814f

In [24]:
stopwords = ['a']
texto = 'juan detesta a maria'
texto = limpiar_texto(texto, stopwords)
print(texto)
arbol = crear_arbol(texto)
rec1(arbol)

2022-02-28 12:55:44 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-02-28 12:55:44 INFO: Use device: cpu
2022-02-28 12:55:44 INFO: Loading: tokenize
2022-02-28 12:55:44 INFO: Loading: mwt
2022-02-28 12:55:44 INFO: Loading: pos
2022-02-28 12:55:45 INFO: Loading: lemma
2022-02-28 12:55:45 INFO: Loading: depparse
2022-02-28 12:55:45 INFO: Loading: ner
2022-02-28 12:55:46 INFO: Done loading processors!


juan detestar maria
  Gender=Masc|  
  Number=Sing   
       |         
  VerbForm=Inf  
       |         
Gender=Fem|Numbe
     r=Sing     



'Gender=Masc|Number=Sing(VerbForm=Inf(Gender=Fem|Number=Sing))'

In [74]:
diccionario = {'camina-ROOT': lambda x: 'CAMINAR(' + str(x) + ')', 
               'juan-nsubj':'j',
               'detesta-ROOT': lambda x, y: 'DETESTAR(' + str(x) + ',' + str(y) + ')', 
               'comer-ccomp': lambda x: 'COMER(' + str(x) + ')', 
               'maria-obj':'m',
               'pizza-obj' : 'p'}

In [75]:
aplicacion(arbol, diccionario)

'DETESTAR(j,m)'